- https://github.com/Arshad221b/Sign-Language-Recognition/blob/master/ASLwithCNN
- https://data-flair.training/blogs/sign-language-recognition-python-ml-opencv/

In [8]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D,MaxPool2D, Dense,Flatten, Dropout
import matplotlib.pyplot as plt

In [2]:
# Loading the dataset
import pandas as pd
import numpy as np

training_set = pd.read_csv('./data/sign_mnist_train.csv')
test_set = pd.read_csv('./data/sign_mnist_test.csv')

In [3]:
SIZE = (28, 28)
CLASSES = dict(zip(range(0, 26), list(map(chr, range(97, 123))))) # labels from A to Z

In [4]:
def preprocess_data(data):
    global SIZE
    labels, images = data[0:, 0], np.array([np.reshape(i[1:], SIZE) for i in data])
    return images, labels

In [5]:
X_train, y_train = preprocess_data(training_set.to_numpy())
X_test, y_test = preprocess_data(test_set.to_numpy())

y_train = np.array(y_train).reshape(-1)
y_test = np.array(y_test).reshape(-1)

y_train = np.eye(len(CLASSES))[y_train]
y_test = np.eye(len(CLASSES))[y_test]

In [6]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((27455, 28, 28), (27455, 26), (7172, 28, 28), (7172, 26))

In [8]:
X_train = X_train.reshape((X_train.shape[0], 28, 28, 1))
X_test = X_test.reshape((X_test.shape[0], 28, 28, 1))

In [9]:
model_params = [
    Conv2D(filters=8, kernel_size=(3,3),strides=(1,1),padding='same',input_shape=(28,28, 1),activation='relu', data_format='channels_last'),
    MaxPool2D(pool_size=(2,2)),
    Conv2D(filters=16, kernel_size=(3,3),strides=(1,1),padding='same',activation='relu'),
    Dropout(0.5),
    MaxPool2D(pool_size=(4,4)),
    Dense(128, activation='relu'),
    Flatten(),
    Dense(26, activation='softmax')
]

params = {
    'loss': 'categorical_crossentropy',
    'optimizer': 'SGD',
    'metrics': ['accuracy']
}

In [10]:
model = Sequential(model_params)
model.summary()  

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 28, 28, 8)         80        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 8)        0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 16)        1168      
                                                                 
 dropout (Dropout)           (None, 14, 14, 16)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 3, 3, 16)         0         
 2D)                                                             
                                                                 
 dense (Dense)               (None, 3, 3, 128)         2

In [11]:
# compile the model with params
model.compile(**params)

# train the model with the train data
# history = model.fit(train_images, train_labels, validation_data=(validation_images, validation_labels), epochs=20)
history = model.fit(X_train, y_train, epochs=50, batch_size=100)

Epoch 1/50
275/275 [==============================] - 16s 40ms/step - loss: 15.7175 - accuracy: 0.0437
Epoch 2/50
275/275 [==============================] - 10s 36ms/step - loss: 3.2335 - accuracy: 0.0434
Epoch 3/50
275/275 [==============================] - 9s 34ms/step - loss: 3.2208 - accuracy: 0.0436
Epoch 4/50
275/275 [==============================] - 11s 39ms/step - loss: 3.2117 - accuracy: 0.0435
Epoch 5/50
275/275 [==============================] - 10s 36ms/step - loss: 3.2038 - accuracy: 0.0438
Epoch 6/50
275/275 [==============================] - 10s 37ms/step - loss: 3.1974 - accuracy: 0.0434
Epoch 7/50
275/275 [==============================] - 10s 38ms/step - loss: 3.1921 - accuracy: 0.0450
Epoch 8/50
275/275 [==============================] - 10s 35ms/step - loss: 3.1881 - accuracy: 0.0445
Epoch 9/50
275/275 [==============================] - 11s 39ms/step - loss: 3.1850 - accuracy: 0.0479
Epoch 10/50
275/275 [==============================] - 11s 41ms/step - loss: 3.182

In [14]:
accuracy = model.evaluate(x=X_test,y=y_test,batch_size=32)
print("Accuracy: ",accuracy[1]*100)

225/225 [==============================] - 2s 8ms/step - loss: 0.2004 - accuracy: 0.9552
Accuracy:  95.52426338195801


In [ ]:
# check the accuracy of the model
from termcolor import colored

batch = 20
loss = 0

if batch > len(X_test):
    print(f"Batch size is greater than the test data size.\nBatch size : {batch}\nTest data size : {len(X_test)}")
    batch = 10

for i in range(batch):
    prediction = model.predict(np.array([X_test[i]]))
    # print(CLASSES[np.argmax(prediction)], "->", CLASSES[np.argmax(y_test[i])])
    predicted_class, actual_class = CLASSES[np.argmax(prediction)], CLASSES[np.argmax(y_test[i])]

    # checks for the classes and sets color's depending on that
    color = 'green' if predicted_class == actual_class else 'red'
    if color == 'red': loss += 1
    print(colored(f'[{i}] Predicted : {predicted_class : <2}|  Expected : {actual_class}', color))

perc = round(100/(batch/(batch-loss)), 3)
print(f"Accuracy {perc}%\nLoss : {100-perc}%")